Анализ текстов на слова

In [176]:
import pandas as pd
import re
import nltk
import pymorphy3
from functools import lru_cache

In [177]:
dataset = pd.read_csv("/Users/a.chervonikov/Desktop/Purify/Purify/purify_ml/data/raw/kartaslovsent.csv", sep=";")
dataset.head()

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19


In [178]:
term_to_tag_dict = dict(zip(dataset['term'], dataset['tag']))

print("Размер словаря:", len(term_to_tag_dict))
print("Пример записи:", list(term_to_tag_dict.items())[0])

test_word = "хороший"
if test_word in term_to_tag_dict:
    print(f"Слово '{test_word}' имеет метку: {term_to_tag_dict[test_word]}")
else:
    print(f"Слово '{test_word}' отсутствует в словаре")

Размер словаря: 46127
Пример записи: ('абажур', 'NEUT')
Слово 'хороший' имеет метку: PSTV


Регулярка на мат

In [179]:
mat_regex = r"""(?iux)(?<![а-яё])(?:
(?:(?:у|[нз]а|(?:хитро|не)?вз?[ыьъ]|с[ьъ]|(?:и|ра)[зс]ъ?|(?:о[тб]|п[оа]д)[ьъ]?|(?:\S(?=[а-яё]))+?[оаеи-])-?)?(?:
  [её](?:б(?!о[рй]|рач)|п[уа](?:ц|тс))|
  и[пб][ае][тцд][ьъ]
).*?|

(?:(?:н[иеа]|(?:ра|и)[зс]|[зд]?[ао](?:т|дн[оа])?|с(?:м[еи])?|а[пб]ч|в[ъы]?|пр[еи])-?)?ху(?:[яйиеёю]|л+и(?!ган)).*?|

бл(?:[эя]|еа?)(?:[дт][ьъ]?)?|

\S*?(?:
  п(?:
    [иеё]зд|
    ид[аое]?р|
    ед(?:р(?!о)|[аое]р|ик)|
    охую
  )|
  бля(?:[дбц]|тс)|
  [ое]ху[яйиеё]|
  хуйн
).*?|

(?:о[тб]?|про|на|вы)?м(?:
  анд(?:[ауеыи](?:л(?:и[сзщ])?[ауеиы])?|ой|[ао]в.*?|юк(?:ов|[ауи])?|е[нт]ь|ища)|
  уд(?:[яаиое].+?|е?н(?:[ьюия]|ей))|
  [ао]л[ао]ф[ьъ](?:[яиюе]|[еёо]й)
)|

елд[ауые].*?|
ля[тд]ь|
(?:[нз]а|по)х
)(?![а-яё])"""

PRONOUNS = ['я', 'ты', 'вы', 'он', 'она', 'оно', 'мы', 'они', 'вас', 'нас', 'их', 'его', 'её']
stopword_set = set(nltk.corpus.stopwords.words('russian'))
stopword_set = stopword_set.union({'это', 'который', 'весь', 'наш', 'свой', 'ещё', 'её', 'ваш', 'также', 'итак'})

Идем в лемматизацию

In [180]:
lemmatizer = pymorphy3.MorphAnalyzer()

@lru_cache(maxsize=10000)
def get_lemma(word):
    return lemmatizer.parse(word)[0].normal_form

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = ' '.join(text.split())
    return text.lower()

def is_pronoun_or_stopword(word):
    return word in PRONOUNS or word in stopword_set

def split_compound_words(word):
    return re.findall(r'[А-Яа-яё]+', word)

word_cache = {}

def get_negative_words(text):
    cleaned_text = clean_text(text)
    print(cleaned_text)
    negative_words = set()
    mat_words = list(re.findall(mat_regex, cleaned_text, re.VERBOSE))
    mat_words = set(split_compound_words("_".join(mat_words)))
    words = split_compound_words(text)
    print(words)
    for word in words:
        if is_pronoun_or_stopword(word):
            continue

        if word in word_cache:
            if word_cache[word] == 'NGTV':
                negative_words.add(word)
            continue

        if word in mat_words:
            word_cache[word] = "NGTV"
            negative_words.add(word)
            continue

        if word == "сука":
            word_cache[word] = "NGTV"
            negative_words.add(word)
            continue

        lemma = get_lemma(word)
        
        if is_pronoun_or_stopword(lemma):
            continue
        
        if lemma in term_to_tag_dict:
            tag = term_to_tag_dict[lemma]
            word_cache[word] = tag
            if tag == 'NGTV':
                negative_words.add(word)
        else:
            word_cache[word] = 'NEUT'
    
    return list(negative_words)


Пример работы

In [182]:
text = '''Который год - в стране царствует смута и развал, властвует произвол финансово- чиновничьей олигархии.
Который год мы ожидаем обещанного благополучия и процветания, получая взамен безудержный рост цен, неплатежи по зарплатам и социальным пособиям, межнациональные войны и конфликты, бандитизм и коррупцию.
Довольно слушать бесконечные обещания и заверения чиновников, терпеть унижения и издевательства обнаглевших "реформаторов".
На попытку Ельцина, с помощью отставки правительства, уйти от ответственности за содеянное - ответим решительным: Ельцина - в отставку !
На попытку Ельцина путем политических рокировок продлить агонию ненавистного антинародного режима - ответим : НЕТ - антинародному курсу !На угрозы президента распустить Государственную Думу, выступающую за изменение курса "реформ ", заявим: Руки прочь от Государственной Думы! Даешь Правительство народного доверия!'''
negative_words = get_negative_words(text)
print("Негативные слова:", negative_words)

который год - в стране царствует смута и развал, властвует произвол финансово- чиновничьей олигархии. который год мы ожидаем обещанного благополучия и процветания, получая взамен безудержный рост цен, неплатежи по зарплатам и социальным пособиям, межнациональные войны и конфликты, бандитизм и коррупцию. довольно слушать бесконечные обещания и заверения чиновников, терпеть унижения и издевательства обнаглевших "реформаторов". на попытку ельцина, с помощью отставки правительства, уйти от ответственности за содеянное - ответим решительным: ельцина - в отставку ! на попытку ельцина путем политических рокировок продлить агонию ненавистного антинародного режима - ответим : нет - антинародному курсу !на угрозы президента распустить государственную думу, выступающую за изменение курса "реформ ", заявим: руки прочь от государственной думы! даешь правительство народного доверия!
['Который', 'год', 'в', 'стране', 'царствует', 'смута', 'и', 'развал', 'властвует', 'произвол', 'финансово', 'чиновнич

Исправление ошибок

In [156]:
errors_dataset = pd.read_csv("/Users/a.chervonikov/Desktop/Purify/Purify/purify_ml/data/raw/orfo_and_typos.L1_5+PHON.csv", sep = ";").iloc[1:, :]
# errors_dataset['weight'] = errors_dataset['weight'].apply(lambda x: '{0:.15f}'.format(float(x)))
errors_dataset.head()

,CORRECT,MISTAKE,WEIGHT
1,болота,балото,0.2652
2,болота,боллото,0.0909
3,болота,болотоэ,0.0909
4,болото,палатаа,0.5000
5,болото,болотл,0.3333


In [119]:
string_weights = errors_dataset[~errors_dataset['weight'].apply(lambda x: str(x).replace('.', '').isdigit())]
string_weights

,correct,error,weight


In [120]:
errors_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90972 entries, 1 to 90972
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   correct  90972 non-null  object
 1   error    90972 non-null  object
 2   weight   90972 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


In [167]:
import pandas as pd
from collections import defaultdict
import re
from Levenshtein import distance as levenshtein_distance
from functools import lru_cache

class SpellChecker:
    def __init__(self, dataset_path, max_distance=2):
        self.correct_words = set()
        self.error_to_correct = defaultdict(list)
        self.max_distance = max_distance
        
        df = pd.read_csv(dataset_path, sep=';', header=None, 
                        names=['correct', 'error', 'weight']).iloc[1:, :]
        
        for _, row in df.iterrows():
            correct = row['correct'].strip().lower()
            error = row['error'].strip().lower()
            weight = float(row['weight'])
            
            self.correct_words.add(correct)
            self.error_to_correct[error].append((correct, weight))
        
        for error in self.error_to_correct:
            self.error_to_correct[error].sort(key=lambda x: x[1], reverse=True)
        
        self.all_known_words = list(self.correct_words) + list(self.error_to_correct.keys())
    
    @lru_cache(maxsize=10000)
    def find_closest_word(self, word):
        if not word:
            return None
            
        word = word.lower()
        
        if word in self.correct_words:
            return word
        if word in self.error_to_correct:
            return self.error_to_correct[word][0][0]
        
        min_distance = float('inf')
        closest_word = None
        
        for known_word in self.all_known_words:
            current_distance = levenshtein_distance(word, known_word)
            if current_distance < min_distance and current_distance <= self.max_distance:
                min_distance = current_distance
                closest_word = known_word
        
        return closest_word
    
    def correct_spelling(self, word):
        word = word.lower().strip()
        
        if hasattr(self, '_spelling_cache') and word in self._spelling_cache:
            return self._spelling_cache[word]
        
        if word in self.correct_words:
            return word
        
        if word in self.error_to_correct:
            correction = self.error_to_correct[word][0][0]
            if not hasattr(self, '_spelling_cache'):
                self._spelling_cache = {}
            self._spelling_cache[word] = correction
            return correction
        
        import time
        start = time.time()
        closest_word = self.find_closest_word(word)
        end = time.time()
        print(f"{end - start}")
        if closest_word:
            if closest_word in self.error_to_correct:
                correction = self.error_to_correct[closest_word][0][0]
            else:
                correction = closest_word
            
            if not hasattr(self, '_spelling_cache'):
                self._spelling_cache = {}
            self._spelling_cache[word] = correction
            return correction
        
        return None
    
    def correct_text(self, text):
        tokens = re.findall(r"(\w+|\W+)", text)
        corrected_tokens = []
        for token in tokens:
            if token.strip() and token[0].isalpha():  
                correction = self.correct_spelling(token)
                if correction is not None:
                    if token[0].isupper():
                        correction = correction.capitalize()
                    corrected_tokens.append(correction)
                else:
                    corrected_tokens.append(token)
            else:
                corrected_tokens.append(token)
        
        return ''.join(corrected_tokens)

checker = SpellChecker("/Users/a.chervonikov/Desktop/Purify/Purify/purify_ml/data/raw/orfo_and_typos.L1_5+PHON.csv")
print(checker.correct_spelling("ашибками"))
text = "Это тествый текст с ашибками и опечатками некрасыво."
print(checker.correct_text(text))

0.024075031280517578
None
0.02206707000732422
9.5367431640625e-07
0.024689197540283203
0.024259090423583984
Это местный текст с ашибками и опечатками некрасиво.


In [169]:
print(checker.correct_spelling("тествый"))

местный


In [154]:
print(checker.correct_spelling("туптй"))  
print(checker.correct_spelling("несуществующееслово"))  
text = "Ты тупкя!"

corrected = checker.correct_text(text)
print(corrected)  

None
None
Ты тупкя!
